In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
# Load the trained,scaler pickle,onehot model
model = load_model('model.h5')

# load the encoder and scaler
with open('label_encoder.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)

with open('onehot_encoding.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)



In [3]:
# Example input data
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [4]:
input_data_updated = {k:[v] for k,v in input_data.items()}
print(input_data_updated)
df = pd.DataFrame(input_data_updated)
df

{'CreditScore': [600], 'Geography': ['France'], 'Gender': ['Male'], 'Age': [40], 'Tenure': [3], 'Balance': [60000], 'NumOfProducts': [2], 'HasCrCard': [1], 'IsActiveMember': [1], 'EstimatedSalary': [50000]}


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [5]:
df['Gender'] = label_encoder_gender.transform(df['Gender'])

In [6]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [7]:
label_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geography_encoded_column = label_encoder_geo.transform(df[['Geography']])

In [ ]:
geography_encoded_df = pd.DataFrame(geography_encoded_column.toarray(),columns=label_encoder_geo.get_feature_names_out(['Geography']))  

In [10]:
geography_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
# combine the onehot encoded data
df = pd.concat([df.drop('Geography',axis=1),geography_encoded_df],axis=1)

In [12]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [13]:
# Scaling the input data
input_scaled = scaler.transform(df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
# predict the chun
prediction = model.predict(input_scaled)
print(prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[[0.03486729]]


In [19]:
prediction_prob = prediction[0][0]

In [20]:
prediction_prob

np.float32(0.034867287)

In [ ]:
if prediction_prob >0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
